In [14]:
from pycocotools.coco import COCO
import skimage.io as io
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image

In [15]:
# dataDir=Path('../input/sartorius-cell-instance-segmentation')
annFile = Path('annotations_all.json')
coco = COCO(annFile)
imgIds = coco.getImgIds()
imgs = coco.loadImgs(imgIds[-3:])
print(imgIds)
print(imgs)

loading annotations into memory...
Done (t=1.20s)
creating index...
index created!
['0030fd0e6378', '0140b3c8f445', '01ae5a43a2ab', '026b3c2c4b32', '029e5b3b89c7', '0323e81d23d9', '03b27b381a5f', '042c17cd9143', '042dc0e561a4', '04928f0866b0', '049f02e0f764', '04cd81dfb245', '053d61766edb', '05c61f0f46b7', '061b49d7dbbb', '06c5740c8b18', '0728b8f39241', '07e9ba109e34', '083b178bb184', '085eb8fec206', '08f52aa2add3', '097d60b2cf71', '0a6ecc5fe78a', '0ba181d412da', '0c5938ac5e3c', '0c90b86742b2', '0cfdeeb0dded', '0df9d6419078', '0e1e2b68fa58', '0e9b40b10de8', '0ea6df67cc77', '0eb1d03df587', '0eca9ecf4987', '0f08d640930b', '0f2a46026693', '0f7d009bc5d3', '100681b6cc7a', '115fad550598', '11c136be56b7', '11c2e4fcac6d', '1242e4ec6ef3', '129f894abe35', '13325f865bb0', '1395c3f12b7c', '144a7a69f67d', '14dbd973a7cd', '15283b194621', '152bcf26456b', '15aeb12e7a83', '16e538bbe8c5', '172334437c88', '174793807517', '17754cb5b287', '17d738f88487', '182c3da676bd', '1874b96fd317', '18d5d665a6af', '194

In [16]:
print(imgIds[0])
i = 0
print(coco.loadImgs(imgIds[i:i+1])[0]['height'])

0030fd0e6378
520


In [ ]:
import numpy as np
from pycocotools import mask as maskUtils
image_instance_masks = {}
for i in range(len(imgIds)):
    img_info  = coco.loadImgs(imgIds[i:i+1])[0]
    image_shape = (img_info['height'], img_info['width'])
    image_id_str = str(img_info['id'])
    
    ann_ids = coco.getAnnIds(imgIds=[image_id_str], iscrowd=False)
    # print(ann_ids)
    
    if not ann_ids:
        image_instance_masks[image_id_str] = np.zeros(image_shape, dtype=np.uint16)

    annotations = coco.loadAnns(ann_ids)
    instance_mask_image = np.zeros(image_shape, dtype=np.uint16)
    instance_id_counter = 1

    for ann in annotations:
        ann_id = ann.get('id', 'N/A')
        rle = ann['segmentation']
        rle = maskUtils.frPyObjects(rle, rle['size'][0], rle['size'][1])
        # print(rle)

        single_cell_binary_mask = None
        single_cell_binary_mask = maskUtils.decode(rle)

        binary_mask = maskUtils.decode(rle)
        instance_mask_image[binary_mask == 1] = instance_id_counter
        instance_id_counter += 1

    image_instance_masks[image_id_str] = instance_mask_image

    # 印出進度
    if (i + 1) % 50 == 0 or (i + 1) == len(imgIds):
        print(f"已處理 {i + 1}/{len(imgIds)} 張圖片")
np.save("all_instance_masks.npy", image_instance_masks)

已處理 50/606 張圖片
已處理 100/606 張圖片
已處理 150/606 張圖片
已處理 200/606 張圖片
